# Read Simulation Data
With the following code, we have developed a method for reading and transforming simulation data from JaamSim. The method consists of two main parts. The first is the `get_WIP` function, which takes data from an ExpressionLogger, and calculates the average estimated throughput, actual throughput, and Work-in-Process (WIP). The second method processes the `RunOutputList` data, transforming it into a more readable dataframe, and also printing out the LaTeX code for the table.

In [1]:
import pandas as pd

#Set full display of columns
pd.set_option('display.max_columns', None)

# Set full width of the output
pd.set_option('display.width', 1000)

# Set full display of rows
pd.set_option('display.max_rows', None)


In [2]:
def get_WIP(file_name):
    # Open the file, skip the first 13 lines
    with open(file_name, 'r') as f:
        lines = f.readlines()[12:]

    # Loop through the lines in the file, due to the format of the file
    # For every 8th row, we must skip the first 4 rows, and keep the next 4
   
    lines_time = []
    lines_e = []
    lines_a = []
    lines_wip = []

    i = 0
    for line in lines:
        if i % 8 != 0 and i % 8 != 1 and i % 8 != 2 and i % 8 != 3:
            lines_time.append(line.strip().split("\t")[0])
            lines_e.append(line.strip().split("\t")[1])
            lines_a.append(line.strip().split("\t")[2])
            lines_wip.append(line.strip().split("\t")[3])
        i += 1

    # Create a dataframe with the data
    df = pd.DataFrame(list(zip(lines_time, lines_e, lines_a, lines_wip)), columns =['Time', 'Actual', 'Estimated', 'WIP'])

    # Convert the columns to numeric
    df['Actual'] = pd.to_numeric(df['Actual'])
    df['Estimated'] = pd.to_numeric(df['Estimated'])
    df['WIP'] = pd.to_numeric(df['WIP'])

    # Calculate the mean of the WIP
    mean = df.groupby('Time').mean()
    
    # Convert from minutes to hours
    mapper = {
        "1020.0" : "17:00",
        "1080.0" : "18:00",
        "1140.0" : "19:00",
        "1200.0" : "20:00"
    }
    
    mean = mean.rename(index=mapper)

    return mean

In [3]:
def get_CT_and_Queue(file_name):
    # The order of the measures in the file
    measures = ['CT_total', 'SD_total', 'CT_FU', 'SD_FU', 'CT_R', 'SD_R', 'CT_N', 'SD_N', 'Queue_Working', 'SQ_Working', 'Queue_Wating_Room', 'SQ_Wating_Room', 'Queue_frontdesk', 'SQ_frontdesk', 'Queue_total_l', 'SQ_total_l', "CT_Working", "SD_Working", 'Queue_total_c', 'SQ_total_c', 'Queue_FU', 'SQ_FU', 'Queue_R', 'SQ_R', 'Queue_N', 'SQ_N']
    
    # Read the file
    with open(file_name, "r") as file:
        lines = file.readlines()[1:]

    print("Read the file")
    line = lines[0].split("\t")[1:]

    # Group two and two values (value and ST. deviation) and create a dictionary with the measures as keys and the values as values
    data_dict = {}
    for i in range(0, len(measures) * 2, 2):
        measure = measures[i // 2]
        value, conf = line[i], line[i+1]
        if measure.startswith('SD'):
            # Add SD as a new column to the corresponding CT
            ct_key = measure.replace('SD', 'CT')
            if ct_key in data_dict:
                data_dict[ct_key]['SD'] = value
        elif measure.startswith('SQ'):
            # Add SD as a new column to the corresponding Queue
            queue_key = measure.replace('SQ', 'Queue')
            if queue_key in data_dict:
                data_dict[queue_key]['SD'] = value
        else:
            # Create new entry for CT measures or others
            data_dict[measure] = {'Mean': value, 'SD': None, 'Confidence': conf}  # Initialize SD and Confidence as None

    # Create a dataframe from the dictionary
    df = pd.DataFrame.from_dict(data_dict, orient='index')
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'Measure'}, inplace=True)

    # Convert columns to numeric and calculate confidence intervals
    df['Mean'] = pd.to_numeric(df['Mean'])
    df['SD'] = pd.to_numeric(df['SD'])
    df['Confidence'] = pd.to_numeric(df['Confidence'])
    df['Confidence_interval_1'] = df['Mean'] - df['Confidence']
    df['Confidence_interval_2'] = df['Mean'] + df['Confidence']
    df  = df.round(3)

    # Create a new column 'Confidence Interval'
    df['Confidence Interval'] = "[" + df['Confidence_interval_1'].astype(str) + ", " + df['Confidence_interval_2'].astype(str) + "]"

    #Drop confidence interval columns
    df.drop(columns=['Confidence_interval_1', 'Confidence_interval_2'], inplace=True)
    df.drop(columns=['Confidence'], inplace=True)

    # Create a datafram for cycle time
    df_cycletime = df.loc[df['Measure'].str.contains('CT')].copy()
    df_cycletime.reset_index(drop=True, inplace=True)

    # Drop last row
    df_cycletime.drop(df_cycletime.tail(1).index, inplace=True)

    # Drop ct prefix
    df_cycletime['Measure'] = df_cycletime['Measure'].str.replace('CT_', '')

    # Rename the measures
    mapper = {
        'total': 'All',
        'FU': 'Follow-up',
        'R': 'Return',
        'N': 'New'
    }

    df_cycletime['Measure'] = df_cycletime['Measure'].replace(mapper)

    # Create a dataframe for queue
    df_queue = df.loc[df['Measure'].str.contains('Queue')].copy()
    df_queue.reset_index(drop=True, inplace=True)

    # Remove Queue prefix
    df_queue['Measure'] = df_queue['Measure'].str.replace('Queue_', '')

    # Rename the measures
    mapper = {
        'Working': 'Waiting for resources',
        'Waiting_Room': 'Waiting Room',
        'frontdesk': 'Front Desk',
        'total_c': 'Total',
        'FU': 'Follow-up',
        'R': 'Return',
        'N': 'New'
    }

    df_queue['Measure'] = df_queue['Measure'].replace(mapper)

    df_queue['Measure'] = df_queue['Measure'].str.replace('total_l', 'Total')
    # remove row 4 /index 3
    df_queue = df_queue.drop(df_queue.index[3])

    return df_cycletime, df_queue

In [4]:
def convert_to_latex(dataframe):
    # Function to convert a dataframe to a latex table
    return dataframe.to_latex(index=False, escape=False)

In [5]:
file_name_dat = "Optimized Schedule.dat"
file_name_wip = "Optimized Schedule-WIP-logger.log"

df_cycletime, df_queue = get_CT_and_Queue(file_name_dat, 1) # Scenario 1
df_wip = get_WIP(file_name_wip,1)

print("Cycle time\n")
print(convert_to_latex(df_cycletime))

print("Queue\n")
print(convert_to_latex(df_queue))

print("WIP\n")
print(df_wip.to_latex())


Read the file
              Measure     Mean      SD Confidence Interval
0            CT_total   87.686  32.476      [87.1, 88.271]
1               CT_FU   74.177  27.860    [73.374, 74.979]
2                CT_R   85.704  29.085     [84.998, 86.41]
3                CT_N  103.848  31.717  [102.955, 104.741]
4       Queue_Working    8.198  10.524      [8.013, 8.383]
5   Queue_Wating_Room   24.941  23.083    [24.596, 25.286]
6     Queue_frontdesk    4.736  11.154       [4.622, 4.85]
7       Queue_total_l   37.875  44.761    [37.394, 38.357]
8          CT_Working   49.810  19.691    [49.619, 50.002]
9       Queue_total_c   37.875  25.877    [37.394, 38.357]
10           Queue_FU   37.666  24.549    [37.004, 38.327]
11            Queue_R   36.424  24.730    [35.861, 36.987]
12            Queue_N   39.996  26.523    [39.247, 40.746]
Cycle time

\begin{tabular}{lrrl}
\toprule
  Measure &    Mean &     SD & Confidence Interval \\
\midrule
    Total &  87.686 & 32.476 &      [87.1, 88.271] \\


In [6]:
file_name_dat = "Optimized Schedule.dat"
file_name_wip = "Optimized Schedule-WIP-logger.log"

df_cycletime, df_queue = get_CT_and_Queue(file_name_dat, 2) # Scenario 2
df_wip = get_WIP(file_name_wip, 2)

print("Cycle time\n")
print(convert_to_latex(df_cycletime))

print("Queue\n")
print(convert_to_latex(df_queue))

print("WIP\n")
print(df_wip.to_latex())

Read the file
              Measure     Mean      SD Confidence Interval
0            CT_total   88.680  32.342     [88.08, 89.281]
1               CT_FU   75.895  28.784    [75.096, 76.694]
2                CT_R   87.857  29.444    [87.094, 88.619]
3                CT_N  102.387  31.474  [101.537, 103.236]
4       Queue_Working    8.758  10.806      [8.565, 8.951]
5   Queue_Wating_Room   25.390  23.571    [25.038, 25.743]
6     Queue_frontdesk    4.756  11.141      [4.642, 4.871]
7       Queue_total_l   38.905  45.518    [38.408, 39.402]
8          CT_Working   49.776  19.732    [49.583, 49.968]
9       Queue_total_c   38.905  26.219    [38.408, 39.402]
10           Queue_FU   39.392  25.736    [38.734, 40.049]
11            Queue_R   38.600  25.258    [37.965, 39.234]
12            Queue_N   38.590  25.909      [37.89, 39.29]
Cycle time

\begin{tabular}{lrrl}
\toprule
  Measure &    Mean &     SD & Confidence Interval \\
\midrule
    Total &  88.680 & 32.342 &     [88.08, 89.281] \\


In [7]:
file_name_dat = "Optimized Schedule.dat"
file_name_wip = "Optimized Schedule-WIP-logger.log"

df_cycletime, df_queue = get_CT_and_Queue(file_name_dat, 3) # Scenario 3
df_wip = get_WIP(file_name_wip, 3)

print("Cycle time\n")
print(convert_to_latex(df_cycletime))

print("Queue\n")
print(convert_to_latex(df_queue))

print("WIP\n")
print(df_wip.to_latex())

Read the file
              Measure     Mean      SD Confidence Interval
0            CT_total   87.830  31.867    [87.229, 88.431]
1               CT_FU   76.144  28.759    [75.293, 76.995]
2                CT_R   87.099  29.286    [86.328, 87.871]
3                CT_N  100.365  30.427   [99.496, 101.234]
4       Queue_Working    8.297  10.567      [8.106, 8.487]
5   Queue_Wating_Room   24.967  23.227    [24.611, 25.323]
6     Queue_frontdesk    4.770  11.144      [4.655, 4.884]
7       Queue_total_l   38.033  44.938    [37.534, 38.533]
8          CT_Working   49.797  19.730     [49.603, 49.99]
9       Queue_total_c   38.033  25.891    [37.534, 38.533]
10           Queue_FU   39.613  25.791     [38.916, 40.31]
11            Queue_R   37.850  25.037    [37.234, 38.467]
12            Queue_N   36.523  24.666    [35.801, 37.245]


Cycle time

\begin{tabular}{lrrl}
\toprule
  Measure &    Mean &     SD & Confidence Interval \\
\midrule
    Total &  87.830 & 31.867 &    [87.229, 88.431] \\
Follow-up &  76.144 & 28.759 &    [75.293, 76.995] \\
   Return &  87.099 & 29.286 &    [86.328, 87.871] \\
      New & 100.365 & 30.427 &   [99.496, 101.234] \\
\bottomrule
\end{tabular}

Queue

\begin{tabular}{lrrl}
\toprule
              Measure &   Mean &     SD & Confidence Interval \\
\midrule
Waiting for resources &  8.297 & 10.567 &      [8.106, 8.487] \\
          Wating_Room & 24.967 & 23.227 &    [24.611, 25.323] \\
           Front Desk &  4.770 & 11.144 &      [4.655, 4.884] \\
                Total & 38.033 & 25.891 &    [37.534, 38.533] \\
            Follow-up & 39.613 & 25.791 &     [38.916, 40.31] \\
               Return & 37.850 & 25.037 &    [37.234, 38.467] \\
                  New & 36.523 & 24.666 &    [35.801, 37.245] \\
\bottomrule
\end{tabular}

WIP

\begin{tabular}{lrrr}
\toprule
{} &  Estimated &  Ac

In [8]:
file_name_dat = "Optimized Schedule.dat"
file_name_wip = "Optimized Schedule-WIP-logger.log"

df_cycletime, df_queue = get_CT_and_Queue(file_name_dat, 4) # Scenario 4
df_wip = get_WIP(file_name_wip, 4)

print("Cycle time\n")
print(convert_to_latex(df_cycletime))

print("Queue\n")
print(convert_to_latex(df_queue))

print("WIP\n")
print(df_wip.to_latex())

Read the file
              Measure    Mean      SD Confidence Interval
0            CT_total  89.935  33.042    [89.321, 90.549]
1               CT_FU  74.937  28.290    [74.081, 75.793]
2                CT_R  94.189  32.107    [93.355, 95.024]
3                CT_N  98.828  30.899     [98.05, 99.606]
4       Queue_Working   8.261  10.767      [8.079, 8.442]
5   Queue_Wating_Room  27.105  24.840    [26.738, 27.472]
6     Queue_frontdesk   4.753  11.148      [4.638, 4.868]
7       Queue_total_l  40.119  46.755    [39.614, 40.623]
8          CT_Working  49.817  19.727    [49.622, 50.012]
9       Queue_total_c  40.119  27.406    [39.614, 40.623]
10           Queue_FU  38.465  25.015    [37.766, 39.164]
11            Queue_R  44.857  28.313    [44.132, 45.581]
12            Queue_N  35.027  24.870    [34.406, 35.648]
Cycle time

\begin{tabular}{lrrl}
\toprule
  Measure &   Mean &     SD & Confidence Interval \\
\midrule
    Total & 89.935 & 33.042 &    [89.321, 90.549] \\
Follow-up & 74.9

In [9]:
file_name_dat = "Optimized Schedule.dat"
file_name_wip = "Optimized Schedule-WIP-logger.log"

df_cycletime, df_queue = get_CT_and_Queue(file_name_dat, 5) # Scenario 5
df_wip = get_WIP(file_name_wip, 5) 

print("Cycle time\n")
print(convert_to_latex(df_cycletime))

print("Queue\n")
print(convert_to_latex(df_queue))

print("WIP\n")
print(df_wip.to_latex())

Read the file
              Measure     Mean      SD Confidence Interval
0            CT_total   87.630  32.806    [87.046, 88.213]
1               CT_FU   74.286  28.279    [73.485, 75.087]
2                CT_R   84.273  29.247     [83.586, 84.96]
3                CT_N  105.472  31.184  [104.551, 106.394]
4       Queue_Working    8.237  10.665       [8.054, 8.42]
5   Queue_Wating_Room   24.815  23.192    [24.473, 25.157]
6     Queue_frontdesk    4.774  11.167       [4.659, 4.89]
7       Queue_total_l   37.827  45.024    [37.347, 38.306]
8          CT_Working   49.803  19.739    [49.611, 49.995]
9       Queue_total_c   37.827  26.119    [37.347, 38.306]
10           Queue_FU   37.834  25.043    [37.183, 38.484]
11            Queue_R   34.899  24.850    [34.356, 35.441]
12            Queue_N   41.665  26.081    [40.847, 42.483]
Cycle time

\begin{tabular}{lrrl}
\toprule
  Measure &    Mean &     SD & Confidence Interval \\
\midrule
    Total &  87.630 & 32.806 &    [87.046, 88.213] \\
